In [1]:
from kernelforge._fchl19 import flocal_kernel, generate_fchl_acsf, generate_fchl_acsf_and_gradients, flocal_kernel_symm, fatomic_local_gradient_kernel
# import kernelforge
# help(kernelforge._fchl19)
import numpy as np
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import time
import tqdm
from tqdm.notebook import tqdm
import pandas as pd
import rdkit
from kernelforge._kernels import solve_cholesky
from sklearn.model_selection import train_test_split
from rdkit import Chem

In [2]:
tqdm.pandas()

In [3]:
def mol_to_arrays(mol):
    """Return (Z, coords) from an RDKit Mol with a conformer.
    
    Z      : (n_atoms,) int (atomic numbers)
    coords : (n_atoms, 3) float (xyz positions)
    """
    conf = mol.GetConformer()
    n = mol.GetNumAtoms()

    Z = np.array([atom.GetAtomicNum() for atom in mol.GetAtoms()], dtype=np.int32)
    coords = np.array([list(conf.GetAtomPosition(i)) for i in range(n)], dtype=np.float64)

    return Z, coords

In [4]:
def pad_rep(rep, n_atoms_max):
    """
    rep : np.ndarray of shape (n_atoms, rep_size)
    n_atoms_max : int, maximum number of atoms across molecules
    
    returns : np.ndarray of shape (n_atoms_max, rep_size)
    """
    n_atoms, rep_size = rep.shape
    if n_atoms > n_atoms_max:
        raise ValueError(f"rep has {n_atoms} atoms, exceeds n_atoms_max={n_atoms_max}")
    
    padded = np.zeros((n_atoms_max, rep_size), dtype=rep.dtype)
    padded[:n_atoms, :] = rep
    return padded

In [5]:
df = pd.read_csv("/home/andersx/dev/laptop-qml/kitchen-sink-paper/datasets/qm7b/qmb7b_data.txt",
                 sep=r"\s+", header=None, names=["filename","energy"]+[f"f{i}" for i in range(1,14)])
df = df[["filename", "energy"]]
elements = [1,6,7,8,16,17]

In [6]:
mols = []
Zs = []
coords = []
for i, row in tqdm(df.iterrows(), total=len(df)):
    mol = Chem.MolFromXYZFile(f"/home/andersx/dev/laptop-qml/kitchen-sink-paper/datasets/qm7b/xyz/{row.filename}")
    mols.append(mol)
    Z, coord = mol_to_arrays(mol)
    coords.append(coord)
    Zs.append(Z)
df["mol"] = mols
df["coords"] = coords
df["Z"] = Zs
df["N"] = df["Z"].apply(len)
df["Z_pad"] = df["Z"].apply(lambda x: np.pad(x, (0, 23 - len(x)), mode="constant"))
df

  0%|          | 0/7211 [00:00<?, ?it/s]

,filename,energy,mol,coords,Z,N,Z_pad
0,dsgdb7njp_00000.xyz,-420.933734,<rdkit.Chem.rdchem.Mol object at 0x75c9f334bd80>,"[[0.99813803, -0.00263872, -0.00464602], [2.09...","[6, 1, 1, 1, 1]",5,"[6, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,dsgdb7njp_00001.xyz,-718.419179,<rdkit.Chem.rdchem.Mol object at 0x75c9f334bd10>,"[[0.99566434, -0.00295079, -0.0064553], [2.524...","[6, 6, 1, 1, 1, 1, 1, 1]",8,"[6, 6, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, ..."
2,dsgdb7njp_00002.xyz,-570.024856,<rdkit.Chem.rdchem.Mol object at 0x75c9f334b840>,"[[0.98946692, 7.55e-05, 0.0], [2.32461012, -0....","[6, 6, 1, 1, 1, 1]",6,"[6, 6, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,dsgdb7njp_00003.xyz,-410.286153,<rdkit.Chem.rdchem.Mol object at 0x75c9f334bbc0>,"[[0.9897241, 0.0, 0.0], [2.20043588, 0.0, 0.0]...","[6, 6, 1, 1]",4,"[6, 6, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,dsgdb7njp_00004.xyz,-868.855422,<rdkit.Chem.rdchem.Mol object at 0x75c9f334bb50>,"[[-0.02685201, 0.87078057, -0.05692871], [-0.7...","[6, 6, 6, 1, 1, 1, 1, 1, 1]",9,"[6, 6, 6, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...
7206,dsgdb7njp_07206.xyz,-1109.600987,<rdkit.Chem.rdchem.Mol object at 0x75c9f23ce500>,"[[1.08422894, 0.27334133, 0.04097233], [2.7096...","[8, 16, 8, 8, 6, 6, 6, 1, 1, 1, 1]",11,"[8, 16, 8, 8, 6, 6, 6, 1, 1, 1, 1, 0, 0, 0, 0,..."
7207,dsgdb7njp_07207.xyz,-1343.513850,<rdkit.Chem.rdchem.Mol object at 0x75c9f23ce5e0>,"[[1.70204007, 0.04370375, 1.59315395], [2.6992...","[16, 6, 7, 6, 6, 6, 7, 1, 1, 1, 1, 1, 1]",13,"[16, 6, 7, 6, 6, 6, 7, 1, 1, 1, 1, 1, 1, 0, 0,..."
7208,dsgdb7njp_07208.xyz,-1330.148845,<rdkit.Chem.rdchem.Mol object at 0x75c9f23ce650>,"[[1.53428342, 0.43993908, 0.9326552], [2.90960...","[16, 6, 7, 6, 6, 6, 7, 1, 1, 1, 1, 1, 1]",13,"[16, 6, 7, 6, 6, 6, 7, 1, 1, 1, 1, 1, 1, 0, 0,..."
7209,dsgdb7njp_07209.xyz,-1355.485597,<rdkit.Chem.rdchem.Mol object at 0x75c9f23ce6c0>,"[[0.79332224, -0.04833491, 0.01449686], [2.472...","[16, 6, 7, 6, 6, 6, 7, 1, 1, 1, 1, 1, 1]",13,"[16, 6, 7, 6, 6, 6, 7, 1, 1, 1, 1, 1, 1, 0, 0,..."


In [7]:
df["rep"] = df.progress_apply(lambda row: generate_fchl_acsf(row["coords"], row["Z"], elements=elements), axis=1)
df

  0%|          | 0/7211 [00:00<?, ?it/s]

,filename,energy,mol,coords,Z,N,Z_pad,rep
0,dsgdb7njp_00000.xyz,-420.933734,<rdkit.Chem.rdchem.Mol object at 0x75c9f334bd80>,"[[0.99813803, -0.00263872, -0.00464602], [2.09...","[6, 1, 1, 1, 1]",5,"[6, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[0.6009690325531315, 2.8554785320884513, 2.76..."
1,dsgdb7njp_00001.xyz,-718.419179,<rdkit.Chem.rdchem.Mol object at 0x75c9f334bd10>,"[[0.99566434, -0.00295079, -0.0064553], [2.524...","[6, 6, 1, 1, 1, 1, 1, 1]",8,"[6, 6, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, ...","[[0.4347161969575047, 2.1157490804590555, 2.07..."
2,dsgdb7njp_00002.xyz,-570.024856,<rdkit.Chem.rdchem.Mol object at 0x75c9f334b840>,"[[0.98946692, 7.55e-05, 0.0], [2.32461012, -0....","[6, 6, 1, 1, 1, 1]",6,"[6, 6, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[0.31427180785860354, 1.4494261266282555, 1.3..."
3,dsgdb7njp_00003.xyz,-410.286153,<rdkit.Chem.rdchem.Mol object at 0x75c9f334bbc0>,"[[0.9897241, 0.0, 0.0], [2.20043588, 0.0, 0.0]...","[6, 6, 1, 1]",4,"[6, 6, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[0.19281621369476667, 0.775654266612096, 0.70..."
4,dsgdb7njp_00004.xyz,-868.855422,<rdkit.Chem.rdchem.Mol object at 0x75c9f334bb50>,"[[-0.02685201, 0.87078057, -0.05692871], [-0.7...","[6, 6, 6, 1, 1, 1, 1, 1, 1]",9,"[6, 6, 6, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, ...","[[0.31941877557119047, 1.4572817596439103, 1.3..."
...,...,...,...,...,...,...,...,...
7206,dsgdb7njp_07206.xyz,-1109.600987,<rdkit.Chem.rdchem.Mol object at 0x75c9f23ce500>,"[[1.08422894, 0.27334133, 0.04097233], [2.7096...","[8, 16, 8, 8, 6, 6, 6, 1, 1, 1, 1]",11,"[8, 16, 8, 8, 6, 6, 6, 1, 1, 1, 1, 0, 0, 0, 0,...","[[0.44576262772008485, 1.0122055178499048, 0.7..."
7207,dsgdb7njp_07207.xyz,-1343.513850,<rdkit.Chem.rdchem.Mol object at 0x75c9f23ce5e0>,"[[1.70204007, 0.04370375, 1.59315395], [2.6992...","[16, 6, 7, 6, 6, 6, 7, 1, 1, 1, 1, 1, 1]",13,"[16, 6, 7, 6, 6, 6, 7, 1, 1, 1, 1, 1, 1, 0, 0,...","[[1.991172120224984e-28, 7.040981678732462e-14..."
7208,dsgdb7njp_07208.xyz,-1330.148845,<rdkit.Chem.rdchem.Mol object at 0x75c9f23ce650>,"[[1.53428342, 0.43993908, 0.9326552], [2.90960...","[16, 6, 7, 6, 6, 6, 7, 1, 1, 1, 1, 1, 1]",13,"[16, 6, 7, 6, 6, 6, 7, 1, 1, 1, 1, 1, 1, 0, 0,...","[[1.3549081002516797e-28, 4.1586942095276264e-..."
7209,dsgdb7njp_07209.xyz,-1355.485597,<rdkit.Chem.rdchem.Mol object at 0x75c9f23ce6c0>,"[[0.79332224, -0.04833491, 0.01449686], [2.472...","[16, 6, 7, 6, 6, 6, 7, 1, 1, 1, 1, 1, 1]",13,"[16, 6, 7, 6, 6, 6, 7, 1, 1, 1, 1, 1, 1, 0, 0,...","[[9.58365938400235e-24, 2.2473615684217876e-11..."


In [8]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

In [9]:
n_train = 4000
df_train = df_train[:n_train].copy()
df_train

,filename,energy,mol,coords,Z,N,Z_pad,rep
2534,dsgdb7njp_02534.xyz,-1524.890090,<rdkit.Chem.rdchem.Mol object at 0x75c9f266d380>,"[[0.99355135, -0.00524581, 0.47850916], [2.444...","[6, 6, 7, 6, 7, 6, 6, 1, 1, 1, 1, 1, 1, 1, 1]",15,"[6, 6, 7, 6, 7, 6, 6, 1, 1, 1, 1, 1, 1, 1, 1, ...","[[0.4414622896613837, 2.1267255618127705, 2.07..."
911,dsgdb7njp_00911.xyz,-1374.823034,<rdkit.Chem.rdchem.Mol object at 0x75c9f25f4ba0>,"[[2.04402366, -0.41643356, -1.26582979], [2.35...","[8, 6, 6, 6, 6, 8, 1, 1, 1, 1, 1, 1, 1, 1]",14,"[8, 6, 6, 6, 6, 8, 1, 1, 1, 1, 1, 1, 1, 1, 0, ...","[[0.47783401130854414, 1.0338252395861323, 0.7..."
6529,dsgdb7njp_06529.xyz,-1638.798033,<rdkit.Chem.rdchem.Mol object at 0x75c9f2593ae0>,"[[1.02783803, 0.04224201, 0.19124826], [2.2493...","[8, 6, 6, 6, 7, 6, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1]",16,"[8, 6, 6, 6, 7, 6, 6, 1, 1, 1, 1, 1, 1, 1, 1, ...","[[2.011741188031152e-19, 4.265540782404292e-09..."
6369,dsgdb7njp_06369.xyz,-1514.129406,<rdkit.Chem.rdchem.Mol object at 0x75c9f258b530>,"[[1.54797962, -1.13402581, -0.63353048], [1.96...","[7, 6, 7, 6, 6, 6, 8, 1, 1, 1, 1, 1, 1, 1, 1]",15,"[7, 6, 7, 6, 6, 6, 8, 1, 1, 1, 1, 1, 1, 1, 1, ...","[[0.6346165204266161, 1.8217786403422715, 1.47..."
6817,dsgdb7njp_06817.xyz,-1289.446694,<rdkit.Chem.rdchem.Mol object at 0x75c9f25af990>,"[[1.18186666, 0.03422594, 0.34983032], [2.5838...","[8, 7, 6, 6, 6, 6, 8, 1, 1, 1, 1, 1]",12,"[8, 7, 6, 6, 6, 6, 8, 1, 1, 1, 1, 1, 0, 0, 0, ...","[[0.39627612976042437, 0.9763494610034825, 0.7..."
...,...,...,...,...,...,...,...,...
3239,dsgdb7njp_03239.xyz,-1751.931899,<rdkit.Chem.rdchem.Mol object at 0x75c9f26a4f90>,"[[0.90897432, -0.02380965, 0.17275185], [2.421...","[6, 6, 6, 6, 6, 8, 6, 1, 1, 1, 1, 1, 1, 1, 1, ...",17,"[6, 6, 6, 6, 6, 8, 6, 1, 1, 1, 1, 1, 1, 1, 1, ...","[[0.4297071132484927, 2.107481311509928, 2.064..."
2383,dsgdb7njp_02383.xyz,-1777.311678,<rdkit.Chem.rdchem.Mol object at 0x75c9f2665540>,"[[0.61047188, -0.37967776, 0.36737949], [2.137...","[6, 6, 6, 7, 6, 6, 6, 1, 1, 1, 1, 1, 1, 1, 1, ...",18,"[6, 6, 6, 7, 6, 6, 6, 1, 1, 1, 1, 1, 1, 1, 1, ...","[[0.4333547855174023, 2.113492854903633, 2.072..."
4311,dsgdb7njp_04311.xyz,-1625.478724,<rdkit.Chem.rdchem.Mol object at 0x75c9f24f27a0>,"[[1.76422052, 0.21069946, 1.27499287], [2.5774...","[8, 6, 6, 6, 6, 6, 6, 1, 1, 1, 1, 1, 1, 1, 1]",15,"[8, 6, 6, 6, 6, 6, 6, 1, 1, 1, 1, 1, 1, 1, 1, ...","[[7.464412184241274e-23, 5.319864874152693e-11..."
1285,dsgdb7njp_01285.xyz,-1812.978468,<rdkit.Chem.rdchem.Mol object at 0x75c9f260aff0>,"[[0.90827971, 0.07318429, -0.00509428], [2.402...","[6, 6, 6, 6, 6, 6, 6, 1, 1, 1, 1, 1, 1, 1, 1, ...",17,"[6, 6, 6, 6, 6, 6, 6, 1, 1, 1, 1, 1, 1, 1, 1, ...","[[0.42679485918369564, 2.102529556831176, 2.06..."


In [10]:
Xt = np.concatenate([[pad_rep(_, 23)] for _ in df_test["rep"]])
Qt = np.concatenate([[Q] for Q in df_test["Z_pad"]])
Nt = df_test["N"].to_numpy()
Yt = df_test["energy"].to_numpy()

In [11]:
X = np.concatenate([[pad_rep(_, 23)] for _ in df_train["rep"]])
Q = np.concatenate([[Q] for Q in df_train["Z_pad"]])
N = df_train["N"].to_numpy()
Y = df_train["energy"].to_numpy()

In [12]:
sigma = 2.5

In [13]:
t0 = time.perf_counter()
K_symm = flocal_kernel_symm(X, Q, N, sigma)
t1 = time.perf_counter()
print(t1-t0)

5.834031324018724


In [14]:
K_solve = K_symm.copy()
for i in range(K_solve.shape[0]):
    K_solve[i,i] += 1e-8

t0 = time.perf_counter()
alphas = solve_cholesky(K_solve, Y)
t1 = time.perf_counter()
print(t1-t0)

0.054954742023255676


In [ ]:
t0 = time.perf_counter()
Kt = flocal_kernel(Xt, X, Qt, Q, Nt, N, sigma)
t1 = time.perf_counter()
print(t1-t0)

In [ ]:
Yp = Kt @ alphas

In [ ]:
rmse = np.sqrt(np.mean(np.square(Yt - Yp)))
res = pearsonr(Yp, Yt)
print(f"RMSE = {rmse}")
print(res)
fig = plt.figure(figsize=(6,6))
plt.xlabel("CCSD(T) Energy [kcal/mol]")
plt.ylabel("Predicted Energy [kcal/mol]")
plt.title("kernelforge cpp FHCL19 QM7B energy learning from Enery")
plt.scatter(Yt, Yp, s=1)